# Importing Libraries

In [1]:
# Importing Libraries
import socket
import threading
import time
import json
import os.path

# Utility Functions

In [2]:
def sendJsonVote(socket, name, province, part, city, party): # create and send json vote
    jasonObj = writeJsonVote(name, province, part, city, party)
    sendJson(socket, jasonObj)

def recvJsonVote(socket): # recv and decode json vote
    res = recvJson(socket)
    return readJsonVote(res)

def sendJson(socket, jasonObj): # send json vote
    jasonStr = json.dumps(jasonObj)
    jasonStr = jasonStr.encode()
    socket.send(jasonStr)
    
def recvJson(socket): # recv json vote
    res = socket.recv(1024)
    res = res.decode()
    res = json.loads(res)
    return res

def readJsonVote(res): # decode json
    return res["Name"], res["Province"], res["Part"], res["City"], res["Party"]

def writeJsonVote(name, province, part, city, party):
    jasonObj = {"Name": name, "Province": province, "Part": part, "City": city, "Party": party}
    return jasonObj

def printJsonVote(res):
    print(res["Name"], res["Province"], res["Part"], res["City"], res["Party"])

def printVote(name, province, part, city, party):
    print(name, province, part, city, party)

def successfullyVoted(vote):
    print("Following Vote Added Successfully")
    printJsonVote(vote)

In [3]:
def whichPart(vote):
    name, province, part, city, party = readJsonVote(vote)
    ind = parts.index(part)
    port = partsPorts[ind]
    
    return socket.gethostname(), port

In [4]:
# function to Save Vote Data as Json Document 
def save_vote_data(jsonObj, filename):
    newObj = read_file_for_append(jsonObj, filename)
    #print("Data : " + str(newObj))
    with open(filename,'w') as f: 
        json.dump(newObj, f, indent=4) 

# function to add to JSON 
def read_file_for_append(jsonObj, filename): 
    with open(filename) as f:
        file_data = json.load(f)
        
        file_data.append(jsonObj)
        
    return file_data

In [5]:
# Vote Counting

# # provinces = ["Punjab", "Sindh", "KPK", "Balochistan"] # Provinces
# # provincesPorts = [20002, 20005, 20008, 20011]
# # parts = ["Lower", "Upper"] # Province Parts
# # parties = ["PTI", "PMLN"] # Election Parties

# # total_provinces = [0, 0, 0, 0]
# # total_parties = [0, 0]
# # total_punjab = [0, 0, 0, 0, 0] # punjab, lower, upper, pti, pmln
# # total_sindh = [0, 0, 0, 0, 0] # punjab, lower, upper, pti, pmln
# # total_kpk = [0, 0, 0, 0, 0] # punjab, lower, upper, pti, pmln
# # total_balochistan = [0, 0, 0, 0, 0] # punjab, lower, upper, pti, pmln

def add_vote(vote):
    global province_name, part_name, parties
    global total_parties
    
    name, province, part, city, party = readJsonVote(vote) 
    
    indParty = parties.index(party)
    
    print("Ind Party : " + str(indParty))
    
    total_parties[0] = total_parties[0] + 1
    total_parties[1 + indParty] = total_parties[1 + indParty] + 1
    
    
    
    
def print_province_stats():
    global province_name, part_name, parties
    global total_parties
    
    print(part_name + ", " + province_name + "\n->Total : " + str(total_parties[0]) + "\n->PTI : " + str(total_parties[1]) + "\n->PMLN : " + str(total_parties[2]))
    

# Multi Threaded Class

In [6]:
# Multi Threading Class
class NewClient(threading.Thread):
    def __init__(self,clientAddress,clientsocket, fileName):
        threading.Thread.__init__(self)
        self.address = clientAddress
        self.socket = clientsocket
        self.file = fileName
        
        print ("New connection added: ", clientAddress)
        
    def run(self):
        print ("\nConnection from : ", self.address)
        
        vote = recvJson(self.socket)
        #printVote(name, province, part, city, party)
        
        
        save_vote_data(vote, self.file)
        add_vote(vote)
        successfullyVoted(vote)
        
        sendJson(self.socket, vote)
        
        
        # Connectinf to Provencial Server
        soc = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        soc.connect((socket.gethostname(), 20002))
        
        sendJson(soc, vote)
        
        res = recvJson(soc)
        
        if(res == soc):
            print("--- Provencial Updated Successfully ---")
        
        print_province_stats()
        

# Main Socket Code

In [7]:
# defining some values
province_name = "Sindh"
part_name = "Upper"
#provinces = ["Punjab", "Sindh", "KPK", "Balochistan"] # Provinces
#provincesPorts = [20002, 20005, 20008, 20011]
#parts = ["Lower", "Upper"] # Province Parts
#partsPorts = [20003, 20004]
parties = ["PTI", "PMLN"] # Election Parties


# Vote Count
total_parties = [0, 0, 0] # total, lower, upper

In [8]:
fileName = r"files\Upper_Sindh_Vote_Data.json"

file_exists = os.path.isfile(fileName) 
 
if not file_exists:
    with open(fileName,'w') as f: # this file will hold the vote data
        json.dump([], f, indent=4)

In [ ]:
#Listening for socket
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
s.bind((socket.gethostname(), 20007))

threadsList = []

print("Listening for Client Requests\n")
while True:
    s.listen(50)
    clientsocket, address = s.accept()
    
    newthread = NewClient(address,clientsocket,fileName) 
    newthread.start() 
    threadsList.append(newthread)
        
    
for thread in threadsList:
    thread.join()


Listening for Client Requests

